In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + 'CQ4Ni_6zhB_MBUMbHmxQ6OlFJjwrZqNW1hhuH7SGS1rxP-CM8StFqZ9en4N0Iaap'}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

    
def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()

    return lyrics

In [3]:
response = request_song_info('dkr','booba')
print(response)

<Response [200]>


In [4]:
song_data = pd.read_csv('Corpus_file.csv') 
song_data.info()
print(song_data.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 5 columns):
Artist         334 non-null object
Title          331 non-null object
Style          331 non-null object
Duration       178 non-null object
lien_lyrics    191 non-null object
dtypes: object(5)
memory usage: 13.2+ KB
['Artist', 'Title', 'Style', 'Duration', 'lien_lyrics']


In [5]:
song_style = song_data["Style"]
song_time = song_data["Duration"]

In [141]:
nb_song = 250
song_title = song_data["Title"].tolist()[:nb_song]
artist_name = song_data["Artist"].tolist()[:nb_song]
song_style = song_data["Style"].tolist()[:nb_song]
song_time = song_data["Duration"].tolist()[:nb_song]


In [142]:
#song_title = ['dkr','champs elysee']
#artist_name = ['booba','sch']
x = []
song_style_select = []
for i in range(len(song_title)):
    response = request_song_info(song_title[i], artist_name[i])
    json = response.json()
    remote_song_info = None

    for hit in json['response']['hits']:
        if artist_name[i].lower() in hit['result']['primary_artist']['name'].lower():
            remote_song_info = hit
            song_style_select.append(song_style[i])
            break

    if remote_song_info:
            song_url = remote_song_info['result']['url']
            x.append(scrap_song_url(song_url))

In [145]:
# Deliminiter strophe
indice_strophe = []
for i in range(len(x)):
    res = []
    test_sub = "\n\n["
    
    test_str= x[i]
    res =[i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    if len(res) == 0:
        test_sub = "\n\n"
        res =[i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
        
    indice_strophe.append(res)

In [147]:
#Delimiteur nom strophe
test_sub = "["
test_str= x[0]
res2 = []
res2 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]

test_sub = "]"
res3 = []
res3 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
print(res2)
print(res3)

[122, 278, 386, 543, 607, 754, 893, 909]
[132, 285, 396, 551, 617, 761, 901, 916]


In [148]:
def getmetadata(test_str):
    liste = []
    test_sub = "["
    test_str = test_str
    res2 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    test_sub = "]"
    res3 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    liste.append(res2)
    liste.append(res3)
    return liste

In [149]:
indice_metadata = []
for i in range(len(x)):
    indice_metadata.append(getmetadata(x[i]))
    

In [150]:
s = x
strophe_name = []
for i in range(len(indice_metadata)):
    name = []
    for k in range(len(indice_metadata[i][0])):
        start = indice_metadata[i][0][k]  
        end = indice_metadata[i][1][k] 
        name.append(s[i][start+1:end])
    strophe_name.append(name)


[['Couplet 1', 'Pont 1', 'Couplet 2', 'Refrain', 'Couplet 3', 'Pont 2', 'Refrain', 'Pont 3'], ['Couplet 1', 'Pre-Refrain', 'Refrain', 'Couplet 2', 'Pre-Refrain', 'Refrain', 'Refrain', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain'], ['Couplet 1', 'Pré-refrain', 'Refrain', 'Couplet 2', 'Pré-refrain', 'Refrain', 'Couplet 3', 'Refrain'], ['Couplet 1', 'Refrain', 'Pont 1', 'Refrain', 'Pont 2', 'Refrain'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Couplet 3'], ['Couplet 1', 'Refrain', 'Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Refrain'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Refrain', 'Outro'], ['Couplet 1', 'Pré-refrain', 'Refrain', 'Couplet 2', 'Pré-refrain', 'Refrain', 'Pont', 'Refrain', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Refrain', 'Post-refrain', 'Outro'], ['Couplet 1', 'Pré-Ref

In [151]:
x_clean = []
for i in range(len(x)):
    if len(indice_strophe[i]) > 0:
        x_clean.append(x[i])

In [153]:
s = x
strophe_clean = []
m = []
for i in range(len(indice_strophe)):
    music = []
    print("######################")
    for j in range(len(indice_strophe[i])-1):
        start = indice_strophe[i][j]  
        end = indice_strophe[i][j+1]
        print("Limite :",start,end)
        data = s[i]
        data_clean = data[start+2:end]
   
        print(i,j)
        if len(strophe_name[i]) > 0:
            data_clean2 = data_clean.split("["+strophe_name[i][j]+"]")
            if len(data_clean2) > 1:
                data_clean2 = data_clean.split("["+strophe_name[i][j]+"]")[1]
        else:
            data_clean2 = data_clean
            
        print("TEXTE : " , data_clean2)
        #print(strophe_name[i][j])
        music.append(data_clean2)
        print("------------")
        
    strophe_clean.append(music)
    



######################
Limite : 120 276
0 0
TEXTE :  
Te sourire dehors à Angoulême
Un chasse-spleen Melchior Paris-Seychelles
Découvrir ton corps depuis la scène
Du sombre décor Lucrèce est reine
------------
Limite : 276 384
0 1
TEXTE :  
On s'était dit des choses
Que l'on ne tiendra pas
Le temps que l'eau se pose
Sur nos ronces lilas
------------
Limite : 384 541
0 2
TEXTE :  
Le soleil s'endort sur Seychelles
Le sable et l'aurore, fleur de sel
Attends-moi je sors, de l'Île aux cerfs
L'orage est de l'or pour se perdre
------------
Limite : 541 605
0 3
TEXTE :   (x4)
I need you so, oh oh
I won't let you go, woh oh
------------
Limite : 605 752
0 4
TEXTE :  
Souffrir la mort pour te plaire
Défendre nos torts et puis se taire
Le désir encore pour l'hiver
Un chaos d'efforts sur Saint Hilaire
------------
Limite : 752 891
0 5
TEXTE :  
On s'était dit des choses
Que l'on ne tiendra pas
Le temps que l'on se pose
Sur nos lauriers mayas

On s'était dit des choses...
------------
Limite : 891

Limite : 139 315
70 1
TEXTE :  Nous deux, un tango à trois temps
Je veux, encore une fois seulement
Un tour de manège
Des feux de joie avant qu'il neige
Des phares, des lumières
De toi et moi avant l'hiver
------------
Limite : 315 635
70 2
TEXTE :  Je veux une grande parade
Une retraite aux flambeaux
Des majorettes, une fontaine
Je veux un drapeau
Une petite mascarade
Un carnaval à Bornéo
Nous deux, un tango à trois temps
Je veux, encore une fois seulement
Un tour de manège
Des feux de joies avant qu'il neige
Des phares, des lumières
De toi et moi avant l'hiver
------------
Limite : 635 725
70 3
TEXTE :  Des feux de joies avant qu'il neige
Des phares, des lumières
De toi et moi avant l'hiver
------------
######################
Limite : 0 121
71 0
TEXTE :  Je suis né sous un ciel de traîne
Entre Hambourg et Cuba
Depuis je me promène
Sur mon nuage à moi
Là où le vent me mène
------------
Limite : 121 240
71 1
TEXTE :  J'ai entendu tant de sirènes chanter ici et là
Que j'ai eu de la pein

------------
Limite : 811 906
144 3
TEXTE :  Chérie, oh chérie
Chérie, oh chérie
Souris-moi je t'en prie
Mon ciel est bleu quand tu souris
------------
######################
Limite : 0 583
145 0
TEXTE :  Dis-moi tout, donne-moi ma chance
Oh Mary Lou
J'ai besoin d'un peu d'espérance
Oh Mary Lou
J'ai besoin de ta présence
Oh Mary Lou
Devant toi je suis sans défense
Oh Mary Lou
Si tu me donnais ta tendresse
Oh Mary Lou
De mon cœur, tu serais la princesse
Oh Mary Lou
Dis-moi tout, donne-moi ma chance
Oh Mary Lou
Pour nous deux, l'amour commence
Oh Mary Lou
Mais que veut dire ton silence
Oh Mary Lou
Tu souris avec insouciance
Oh Mary Lou
J'ai eu tort de vouloir te plaire
De t'aimer comme un fou
Mon Amour ne t'intéresse guère
Adieu, Mary Lou
Oh Mary Lou
Oh Mary Lou
Oh Mary Lou ...
------------
######################
Limite : 0 267
146 0
TEXTE :  Eddie réveille-toi, tu dors depuis des heures
Crois-moi il est bien temps de songer au labeur
Au lieu d'aller traîner dans les boîtes de nuit
Et de

In [154]:
"""test_sub = "\n"
for i in range(len(substring)):
    test_str= substring[i]
    r = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    res4.append(r)
print(res4)"""

'test_sub = "\n"\nfor i in range(len(substring)):\n    test_str= substring[i]\n    r = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]\n    res4.append(r)\nprint(res4)'

In [156]:
import xml.etree.ElementTree as ET
import xml.etree.cElementTree as ET

In [157]:

root = ET.Element("root")
for i in range(len(strophe_clean)):
    print(i," || ",song_title[i]," || ",artist_name[i])
    doc = ET.SubElement(root, "doc",name =song_title[i],artist =artist_name[i],style=song_style[i],duree="",genre="")
    for j in range(len(strophe_clean[i])-1):
        if len(strophe_name[i]) >0:
            ET.SubElement(doc, "lg",name=strophe_name[i][j], type="strophe").text = strophe_clean[i][j]
        else:
            ET.SubElement(doc, "lg",name="", type="strophe").text = strophe_clean[i][j]

tree = ET.ElementTree(root)
tree.write("Lyric_song.xml", encoding="UTF-8")

0  ||  Paris-Seychelles  ||  Julien Doré
1  ||  Le chant des sirènes  ||  Fréro Delavega
2  ||  Jour 1  ||  Louane
3  ||  Les filles d'aujourd'hui  ||  Vianney
4  ||  Color Gitano  ||  Kendji Girac
5  ||  Dernière Danse  ||  Indila
6  ||  L'attrape-rêves  ||  Christophe Maé
7  ||  A bouche que veux-tu  ||  Brigitte
8  ||  Paradis secret  ||  Jenifer
9  ||  Un Jour Au Mauvais Endroit  ||  Calogero
10  ||  Rien qu'une fois  ||  Keen'V
11  ||  Je suis chez moi  ||  Black M
12  ||  Sapés comme jamais (Pilule Bleue)  ||  Maître Gims
13  ||  Oui ou non  ||  Angèle
14  ||  VIDEOCLUB  ||  En nuit
15  ||  Ne m’invite pas  ||  Poupie
16  ||  Défait  ||  Julien Granel
17  ||  Casanova  ||  Soprano
18  ||  Maria Maria  ||  Kendji Girac
19  ||  Les matins  ||  Angèle
20  ||  Clown  ||  Soprano
21  ||  En feu  ||  Soprano
22  ||  Zaho  ||  Tant de choses
23  ||  Souf  ||  Tout va bien
24  ||  Alors on danse  ||  Stromae
25  ||  Papaoutai  ||  Stromae
26  ||  tous les mêmes  ||  Stromae
27  ||  Basiq

In [158]:

"""
import json

data = {}
data['music'] = []
for i in range(len(strophe_clean)):
    for j in range(len(strophe_clean[i])-1):
        data['music'].append({
            'artist': artist_name[i],
            'title' : song_title[i],
            'duree': song_time[i],
            'style': song_style[i],
            'lyric': strophe_clean[i][j]
        })


with open('data.json', 'w') as outfile:
    json.dump(data, outfile)
    """

"\nimport json\n\ndata = {}\ndata['music'] = []\nfor i in range(len(strophe_clean)):\n    for j in range(len(strophe_clean[i])-1):\n        data['music'].append({\n            'artist': artist_name[i],\n            'title' : song_title[i],\n            'duree': song_time[i],\n            'style': song_style[i],\n            'lyric': strophe_clean[i][j]\n        })\n\n\nwith open('data.json', 'w') as outfile:\n    json.dump(data, outfile)\n    "